In [1]:
import json
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Environment, Experiment
from azureml.data import FileDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.data.datapath import DataPath, DataPathComputeBinding
from azureml.pipeline.core import Pipeline, PipelineData, PipelineParameter
from azureml.core.compute import ComputeTarget, AmlCompute


print(azureml.core.VERSION)
version = dict(zip(['major','minor','patch'], azureml.core.VERSION.split('.')))
ws = Workspace.from_config()

1.10.0


In [2]:
if int(version['major']) >= 1: 
    if int(version['minor']) == 10:
        from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep        
    else:
        from azureml.contrib.pipeline.steps import ParallelRunConfig, ParallelRunStep 

In [3]:
compute_name = "aml-compute1"
vm_size = "STANDARD_DS1_v2"
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found compute target: ' + compute_name)
else:
    print('compute target not found, refer to 02_create_compute_cluster to create compute target...')

Found compute target: aml-compute1


In [17]:
datastore_name = 'godzilla'
if datastore_name in ws.datastores:
    datastore = ws.datastores[datastore_name]
    if datastore and type(datastore) is Datastore: 
        print('Found datastore: ' + datastore_name)
else: 
    print('datastore not found...')

images_dataset_name = 'images_partition'
datapath_on_datastore = DataPath(datastore=datastore, path_on_datastore='images')
# path_on_datastore = datastore.path('images')
# input_images_dataset = Dataset.File.from_files(path=path_on_datastore, validate=False)

In [18]:
type(datapath_on_datastore)

azureml.data.datapath.DataPath

In [19]:
output_dir = PipelineData(name="scores", 
                          datastore=ws.get_default_datastore(), 
                          output_path_on_compute="batchscoring/results")

In [20]:
output_dir

$AZUREML_DATAREFERENCE_scores

# pipeline parameters and datapath


In [27]:
# kv = ws.get_default_keyvault()
# print(len(kv.get_secret(pipeline_kv_readapi)))
pipeline_inpart = PipelineParameter(name="pipeline_inpart", default_value='2020/07/28')
pipeline_kv_customimg = PipelineParameter(name="pipeline_kv_customimg", default_value='api-custom-vision')
pipeline_kv_readapi = PipelineParameter(name="pipeline_kv_readapi", default_value='api-readapi')
pipeline_datapath_param = PipelineParameter(name='pipeline_datapath', default_value=datapath_on_datastore)
datapath_input = (pipeline_datapath_param, DataPathComputeBinding(mode='mount'))


In [28]:
print(pipeline_inpart)
print(pipeline_datapath_param)
print(datapath_input)

PipelineParameter_Name:pipeline_inpart_Default:2020/07/28
PipelineParameter_Name:pipeline_datapath_Default:<azureml.data.datapath.DataPath object at 0x000001BB5F0E36D0>
(<azureml.pipeline.core.graph.PipelineParameter object at 0x000001BB5F585FD0>, <azureml.data.datapath.DataPathComputeBinding object at 0x000001BB5F585310>)


# python envinronment configuration 

In [24]:
# conda dependencies 
env_name = 'MAG-ParallelRunEnv'
print(env_name in ws.environments)
if env_name in ws.environments: 
    env = ws.environments.get(env_name)
    if env and type(env) is Environment: 
        print('Found environment: ' + env_name)
else: 
    print('environment not found, refer to 01_config_notebook to register envinroment...')


True
Found environment: MAG-ParallelRunEnv


# parallel run config and step

In [29]:
parallelrunconfig = ParallelRunConfig(
    environment=env, 
    entry_script='minibatch_process.py', 
    error_threshold=1,
    output_action='append_row', 
    compute_target=compute_target, 
    node_count=1, 
    process_count_per_node=2,
    mini_batch_size='2',
    source_directory='scripts', 
    description='description of batch step config',
    logging_level='INFO'
)


#parallelrunconfig('script', 'other stuff')
parallelrunstep = ParallelRunStep(
    name='cv-detection-batch-dataset-step', 
    parallel_run_config=parallelrunconfig, 
    inputs=[datapath_input],
    # inputs=[DatasetConsumptionConfig('dataset_param_config', pipeline_dataset_param).as_mount()], 
    # inputs=[ Dataset.File.from_files((godzilla_datastore, 'images')).as_named_input('anpr_images').as_mount()],
    # inputs=[]
    side_inputs=[], 
    output=output_dir,
    arguments=['--input_partition', pipeline_inpart, '--kv_customimage', pipeline_kv_customimg, '--kv_readapi',)pipeline_kv_readapi], 
    allow_reuse=False
)


# ('config', 'inputs as mount', 'arguments passing in pipeline args')

Exception: Step input must be of any type: dict_keys([<class 'azureml.data.tabular_dataset.TabularDataset'>, <class 'azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset'>, <class 'azureml.data.file_dataset.FileDataset'>, <class 'azureml.pipeline.core.pipeline_output_dataset.PipelineOutputFileDataset'>]), found <class 'tuple'>

In [11]:
# data transfer step 

to move score data from out temp storage to blog storage

SyntaxError: invalid syntax (<ipython-input-11-06adc92525fb>, line 3)

# prepare pipeline 

In [12]:
pipeline = Pipeline(workspace=ws, steps=[parallelrunstep])
pipeline_run = Experiment(ws, 'MAG-batch-paramdatapath').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

NameError: name 'parallelrunstep' is not defined

In [13]:
published_pipeline = pipeline_run.publish_pipeline(
    name='MAG-batchscore-datapath', 
    description='published pipeline with datapath param 15:30pm', 
    version='1.0', 
    continue_on_step_failure=True)

NameError: name 'pipeline_run' is not defined

In [14]:
published_pipeline.endpoint

NameError: name 'published_pipeline' is not defined